In [9]:
import math

# The sigmoid activation function.
def activation_function(x):
    return 1 / (1 + math.exp(-x))

# The derivative of the sigmoid activation function.
def activation_derivative(output):
    return output * (1 - output)

In [10]:
class Neuron:
    def __init__(self, name):
        self.name = name
        self.inputs = []
        self.outputs = []
        self.sum = 1   # Set sum and value to 1 (mostly for bias neurons).
        self.output = 1

    # Use our input synapses to set our value.
    def set_value(self):
        # Don't change the bias neuron values.
        if self.name == 'Bias':
            return

        # Add up the inputs.
        self.sum = 0
        for synapse in self.inputs:
            self.sum += synapse.from_neuron.output * synapse.weight

        # Run through the activation function.
        self.output = activation_function(self.sum)

In [11]:
class Synapse:
    def __init__(self, from_neuron, to_neuron, weight):
        self.from_neuron = from_neuron
        self.to_neuron = to_neuron
        self.weight = weight

        if from_neuron is not None:  # This happens if this is a bias neuron.
            from_neuron.outputs.append(self)
        if to_neuron is not None:    # This should not happen.
            to_neuron.inputs.append(self)

In [12]:
import random

# layer_sizes is a list holding the number of
# neurons (not counting biases) in the layers.
class NeuralNet:
    def __init__(self, layer_sizes):
        # Build the layers.
        self.all_layers = []
        num_layers = len(layer_sizes)
        for i in range(num_layers):
            add_bias = i < num_layers - 1  # No bias for after the output layer.
            self.all_layers.append(self.build_layer(layer_sizes[i], f'Neuron_{i}', add_bias))

        # Make shortcuts to the input and output layers.
        self.input_layer = self.all_layers[0]
        self.output_layer = self.all_layers[-1]

        # Make synapses between layers.
        for layer_num in range(num_layers - 1):
            self.build_synapses(
                self.all_layers[layer_num],
                self.all_layers[layer_num + 1])

    # Make a list of {num_neurons} neurons.
    # Give them names {base_name}_{i} where {i} is the neuron number.
    # If {add_bias} is True, add a bias neuron.
    def build_layer(self, num_neurons, base_name, add_bias):
        layer = []

        # Create the neurons.
        for i in range(num_neurons):
            layer.append(Neuron(f'{base_name}_{i}'))

        # Add the bias neuron.
        if add_bias:
            layer.append(Neuron(f'Bias'))
        return layer

    # Make links connecting two neuron layers.
    def build_synapses(self, from_layer, to_layer):
        for from_neuron in from_layer:
            for to_neuron in to_layer:
                # No synapses lead into a bias neuron.
                if to_neuron.name != 'Bias':
                    Synapse(from_neuron, to_neuron, 0.5)

    # Run the network on these inputs.
    # Return a list of output values.
    def evaluate(self, input_values):
        # Make sure we have the right number of input values.
        # (The +1 is for the bias neuron in the input layer.)
        if len(input_values) + 1 != len(self.input_layer):
            raise TypeError('Number of input values does not match the size of the input layer')

        # Set the input layer values.
        for i in range(len(input_values)):
            self.input_layer[i].output = input_values[i]

        # Update the network.
        for i in range(1, len(self.all_layers)):
            # Update layer i.
            for neuron in self.all_layers[i]:
                neuron.set_value()

        # Return the output layer's results.
        results = [neuron.output for neuron in self.output_layer]
        return results

    # Print the neuron names and synapse weights.
    def dump(self):
        for layer_num in range(len(self.all_layers)):
            print(f'Layer {layer_num}:')
            for neuron in self.all_layers[layer_num]:
                print(f'    {neuron.name}: ', end='')
                for synapse in neuron.outputs:
                    print(f'{synapse.weight:>7.2f} ', end='')
                print()

    # Randomize the network's synapse weights.
    def randomize(self, min_weight, max_weight):
        for layer in self.all_layers:
            for neuron in layer:
                for synapse in neuron.inputs:
                    synapse.weight = random.uniform(min_weight, max_weight)

    # Train the network for one epoch.
    # {input_list} is a list of sets of inputs.
    # {target_list} is a list holding corresponding sets of target outputs.
    # Return the average error over this epoch.
    def train_one_epoch(self, input_list, target_list, learning_rate):
        # Randomize the input and target lists.
        combined = list(zip(input_list, target_list))
        random.shuffle(combined)
        input_list, target_list = zip(*combined)

        # Process the epoch.
        total_error = 0
        for i in range(len(input_list)):
            # Evaluate this input.
            self.evaluate(input_list[i])

            # Use back propagation to adjust synapse values.
            total_error += self.backpropagate(target_list[i], learning_rate)

        return total_error / len(input_list)

    # Train the network for {num_epochs} epochs.
    def train(self, inputs, targets, learning_rate, num_epochs):
        for i in range(num_epochs):
            epoch_error = self.train_one_epoch(inputs, targets, learning_rate)
        return epoch_error

    # Calculate output neuron errors.
    # Use the half square error formula.
    def calculate_error(self, targets):
        total_error = 0

        # Loop through the output layer's synapses by index.
        for i in range(len(self.output_layer)):
            # Get the neuron by index.
            neuron = self.output_layer[i]

            # Calculate the difference between the
            # synapse's output and the target output.
            diff = (targets[i] - neuron.output)

            # Add to the total error.
            total_error += diff * diff
        return total_error / len(self.output_layer)

    # Use back propagation to adjust synapse values.
    # https://builtin.com/machine-learning/backpropagation-neural-network
    # That article used an activation function f(x) = x,
    # so its derivative f'(x) is 1 for all x.
    # We use the sigmoid function instead, so we need to use the derivative
    # of the sigmoid function where the article uses f'.
    # Return the total error.
    def backpropagate_deltas(self, targets):
        # Calculate output neuron deltas.
        for neuron_num in range(len(self.output_layer)):
            neuron = self.output_layer[neuron_num]
            neuron.delta = (targets[neuron_num] - neuron.output) * activation_derivative(neuron.output)

        # Calculate delta for the neurons in earlier layers.
        num_layers = len(self.all_layers)
        for layer_num in reversed(range(num_layers - 1)):
            # Find the deltas for neurons in layer layer_num.
            for neuron_num in range(len(self.all_layers[layer_num])):
                # Calculate the delta for this neuron.
                # delta_0 = w * delta_1 * f'(z)
                neuron = self.all_layers[layer_num][neuron_num]
                neuron.delta = 0

                # Loop over this neuron's outgoing synapses.
                for synapse in neuron.outputs:
                    neuron.delta += \
                        synapse.weight * \
                        synapse.to_neuron.delta * \
                        activation_derivative(neuron.output)

    # Use the neuron deltas to update the synapse weights.
    def update_weights(self, learning_rate):
        # Loop through all layers except the output layer.
        for layer in self.all_layers[:-1]:
            # Loop through the neurons in this layer.
            for neuron in layer:
                # Update this neuron's outgoing synapses.
                for synapse in neuron.outputs:
                    synapse.weight += \
                        learning_rate * \
                        neuron.output * \
                        synapse.to_neuron.delta

    # Perform one round of training.
    # Use backpropagation to set the neuron deltas.
    # Then update synapse weights.
    def backpropagate(self, targets, learning_rate):
        self.backpropagate_deltas(targets)
        self.update_weights(learning_rate)
        return self.calculate_error(targets)

In [13]:
# Functions to draw the network.
NEURON_RADIUS = 20

# Position the neurons for drawing.
def position_neurons(network, canvas):
    # Get the canvas's dimensions.
    canvas.winfo_toplevel().update()
    width = canvas.winfo_width()
    height = canvas.winfo_height()

    # Find the maximum number of neurons in any layer.
    max_neurons = len(max(network.all_layers, key=len))

    # Calculate some spacing values.
    margin = NEURON_RADIUS + 5
    neuron_gap = (height - 2 * margin) / (max_neurons - 1)  # Spacing between neurons in a layer.
    layer_gap = (width - 2 * margin) / (len(network.all_layers) - 1)  # Spacing between layers.

    # Position the neurons.
    x = margin
    for layer in network.all_layers:
        neuron_gap = (height - 2 * margin) / (len(layer) - 1)  # Spacing between neurons in a layer.
        y = (height - neuron_gap * (len(layer) - 1)) / 2
        for neuron in layer:
            neuron.x = x
            neuron.y = y
            y += neuron_gap
        x += layer_gap

# Draw the network's synapses.
def draw_synapses(network, canvas):
    # Inputs.
    for layer in network.all_layers:
        for neuron_num in range(len(layer)):
            from_neuron = layer[neuron_num]
            for synapse in from_neuron.outputs:
                draw_synapse(synapse, canvas, neuron_num)

# Draw a synapse.
def draw_synapse(synapse, canvas, neuron_num):
    from_neuron = synapse.from_neuron
    to_neuron = synapse.to_neuron

    # Draw the line.
    canvas.create_line(
        from_neuron.x, from_neuron.y,
        to_neuron.x, to_neuron.y,
        fill='blue')

    # Draw the weight text.
    x = (5 * from_neuron.x + 1 * to_neuron.x) / 6
    y = (5 * from_neuron.y + 1 * to_neuron.y) / 6
    text_tag = canvas.create_text(x, y, text=f'{synapse.weight:.2f}')

    # Clear an area for the text.
    bounds = canvas.bbox(text_tag)  # Returns (x1, y1, x2, y2)
    canvas.create_oval(*bounds, fill='white', outline='white')

    # Lift the label above the cleared oval.
    canvas.tag_raise(text_tag)

# Draw the network's neurons.
def draw_neurons(network, canvas):
    for layer in network.all_layers:
        for neuron in layer:
            draw_neuron(neuron, canvas)

# Draw a neuron.
def draw_neuron(neuron, canvas):
    width = 1
    if neuron.name.startswith('Bias'):
        width = 3

    color = 'silver'  # For uncomitted neurons.
    if neuron.name.startswith('Bias'):
        color = 'pink'
    elif neuron.output < 0.33:
        color = 'white'
    elif neuron.output > 0.67:
        color = 'lightgreen'

    canvas.create_oval(
        neuron.x - NEURON_RADIUS, neuron.y - NEURON_RADIUS,
        neuron.x + NEURON_RADIUS, neuron.y + NEURON_RADIUS,
        fill=color, outline='black', width=width)
    # Draw the neuron's value.
    canvas.create_text(neuron.x, neuron.y, text=f'{neuron.output:.2f}')
    # Draw the neuron's sum and value.
    #canvas.create_text(neuron.x, neuron.y, text=f'{neuron.sum:.2f}\n{neuron.output:.2f}')

# Make a simple drawing of the network.
def draw_network(network, canvas):
    canvas.delete('all')

    # Position the neurons.
    position_neurons(network, canvas)

    # Draw the synapses.
    draw_synapses(network, canvas)

    # Draw the neurons.
    draw_neurons(network, canvas)

In [14]:
# Create a label and text field. Return the text field.
def make_field(window, label_text, label_width, default_text, pady):
    frame = tk.Frame(window)
    frame.pack(side=tk.TOP, pady=pady)

    label = tk.Label(frame, text=label_text, width=label_width, anchor=tk.W)
    label.pack(side=tk.LEFT)

    text = tk.StringVar()
    text.set(default_text)
    entry = tk.Entry(frame, width=7,
        textvariable=text, justify=tk.RIGHT)
    entry.pack(side=tk.LEFT)

    return entry

In [15]:
import tkinter as tk

# Geometry constants.
WINDOW_WID = 910
WINDOW_HGT = 610
FRAME1_WID = 150
PADX = 5
NETWORK_CANVAS_WID = WINDOW_WID - FRAME1_WID - 4 * PADX
NETWORK_CANVAS_HGT = WINDOW_HGT - 2 * PADX

class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.network = None

        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('network_topology')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry(f'{WINDOW_WID}x{WINDOW_HGT}')

        # Build the UI.
        self.build_ui()

        # Load the data.
        self.load_data()

        # Draw the network.
        self.redraw_network()

        # Display the window.
        self.window.focus_force()
        self.window.mainloop()

    # Make changes here...
    # Build an odd/even test network.
    def load_data(self):
        num_hidden_neurons = int(self.num_hidden_neurons_text.get())

        # Build the network.
        self.network = NeuralNet([3, num_hidden_neurons, 2])

        # Display the network.
        #self.network.dump()

    # Set the weights for the synapses in all layers.
    def set_synapse_weights(self, all_layers, weights):
        for layer_num in range(len(all_layers)):
            layer = all_layers[layer_num]
            for neuron_num in range(len(layer)):
                neuron = layer[neuron_num]
                for synapse_num in range(len(neuron.outputs)):
                    synapse = neuron.outputs[synapse_num]
                    synapse.weight = weights[layer_num][neuron_num][synapse_num]

    def build_ui(self):
        # Make controls to define the network.
        frame1 = tk.Frame(self.window, width=FRAME1_WID)
        frame1.pack(side=tk.LEFT, expand=False, fill=tk.Y, padx=PADX)
        frame1.pack_propagate(False)

        frame2 = tk.Frame(self.window)
        frame2.pack(side=tk.LEFT, expand=True, fill=tk.BOTH, padx=PADX)

        # Number of neurons in the hidden layer.
        self.num_hidden_neurons_text = make_field(frame1, 'Hidden Neurons:', 13, '3', (0,5))

        # Build button.
        build_button = tk.Button(frame1, width=11, text='Build', command=self.build)
        build_button.pack(side=tk.TOP, pady=(2,20))

        # Min and max synapse weights.
        self.min_weight_text = make_field(frame1, 'Min Weight:', 13, '-10', (0,5))
        self.max_weight_text = make_field(frame1, 'Max Weight:', 13, '10', (0,5))

        # Randomize button.
        randomize_button = tk.Button(frame1, width=11, text='Randomize', command=self.randomize)
        randomize_button.pack(side=tk.TOP, pady=(2,20))

        # Num epochs.
        self.epochs_text = make_field(frame1, 'Num Epochs:', 13, '10000', (0,5))

        # Epochs per tick.
        self.epochs_per_tick_text = make_field(frame1, 'Epochs Per Tick:', 13, '1000', (0,5))

        # Learning rate.
        self.learning_rate_text = make_field(frame1, 'Learning Rate:', 13, '0.5', (0,5))

        # Train button.
        train_button = tk.Button(frame1, width=11, text='Train', command=self.train)
        train_button.pack(side=tk.TOP, pady=(2,10))
        
        # Error Listbox.
        error_label = tk.Label(frame1, text='Epoch Errors:', width=12, anchor=tk.W)
        error_label.pack(side=tk.TOP)

        error_frame = tk.Frame(frame1)
        error_frame.pack(side=tk.TOP)
        scrollbar = tk.Scrollbar(error_frame, orient="vertical")
        self.error_listbox = tk.Listbox(error_frame, width=20, yscrollcommand=scrollbar.set)
        self.error_listbox.pack(side=tk.LEFT)
        scrollbar.config(command=self.error_listbox.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Checkbuttons.
        check_frame = tk.Frame(frame1, width=200)
        check_frame.pack(side=tk.TOP, pady=(20,0))

        self.check0_value = tk.IntVar()
        check0 = tk.Checkbutton(check_frame, variable=self.check0_value,
            onvalue=1, offvalue=0, command=self.redraw_network)
        check0.pack(side=tk.LEFT)

        self.check1_value = tk.IntVar()
        check1 = tk.Checkbutton(check_frame, variable=self.check1_value,
            onvalue=1, offvalue=0, command=self.redraw_network)
        check1.pack(side=tk.LEFT)

        self.check2_value = tk.IntVar()
        check2 = tk.Checkbutton(check_frame, variable=self.check2_value,
            onvalue=1, offvalue=0, command=self.redraw_network)
        check2.pack(side=tk.LEFT)

        # Labels to display results.
        self.even_value = tk.StringVar()
        self.even_value.set('Even = True')
        even_label = tk.Label(frame1, textvariable=self.even_value, height=1)
        even_label.pack(side=tk.TOP)

        self.odd_value = tk.StringVar()
        self.odd_value.set('Odd = False')
        odd_label = tk.Label(frame1, textvariable=self.odd_value, height=1)
        odd_label.pack(side=tk.TOP)

        # Frame 2.
        # Network canvas.
        self.canvas = tk.Canvas(frame2, bg='white',
            borderwidth=0, highlightthickness=0, relief=tk.SUNKEN,
            width=NETWORK_CANVAS_WID, height=NETWORK_CANVAS_HGT)
        self.canvas.pack(side=tk.LEFT, anchor=tk.NW)

    # Redraw the network after the user changed a checkbutton.
    def redraw_network(self):
        # Evaluate with the current checkbutton values.
        input_values = [
            self.check0_value.get(),
            self.check1_value.get(),
            self.check2_value.get(),
        ]
        self.network.evaluate(input_values)

        # Display the results textually.
        if self.network.output_layer[0].output < 0.33:
            self.odd_value.set('Odd = False')
        elif self.network.output_layer[0].output > 0.67:
            self.odd_value.set('Odd = True')
        else:
            self.odd_value.set('??????')

        if self.network.output_layer[1].output < 0.33:
            self.even_value.set('Even = False')
        elif self.network.output_layer[1].output > 0.67:
            self.even_value.set('Even = True')
        else:
            self.even_value.set('??????')

        # Draw the network.
        draw_network(self.network, self.canvas)

    def kill_callback(self):
        self.window.destroy()

    # Build the network.
    def build(self):
        # Clear previous results.
        self.error_listbox.delete(0, tk.END)

        # Build and draw the network.
        self.load_data()
        self.redraw_network()

    # Randomize the network's synapse weights.
    def randomize(self):
        # Clear previous results.
        self.error_listbox.delete(0, tk.END)
        self.num_epochs_trained = 0

        # Randomize the synapses.
        min_weight = float(self.min_weight_text.get())
        max_weight = float(self.max_weight_text.get())
        self.network.randomize(min_weight, max_weight)
        self.redraw_network()

    # Train the network.
    def train(self):
        # Get parameters.
        num_epochs = int(self.epochs_text.get())
        epochs_per_tick = int(self.epochs_per_tick_text.get())
        learning_rate = float(self.learning_rate_text.get())
        inputs = [
            [0, 0, 0],
            [0, 0, 1],
            [0, 1, 0],
            [0, 1, 1],
            [1, 0, 0],
            [1, 0, 1],
            [1, 1, 0],
            [1, 1, 1],
        ]
        targets = [
            [0, 1],
            [1, 0],
            [1, 0],
            [0, 1],
            [1, 0],
            [0, 1],
            [0, 1],
            [1, 0],
        ]
        max_epoch = self.num_epochs_trained + num_epochs # Stop after this epoch.
        self.tick(inputs, targets, learning_rate, max_epoch, epochs_per_tick)

    # Train for {num_epochs} in batches of {epochs_per_tick}.
    def tick(self, inputs, targets, learning_rate, max_epoch, epochs_per_tick):
        # Train for {epochs_per_tick} epochs.
        epoch_error = self.network.train(inputs, targets, learning_rate, epochs_per_tick)

        # Display the latest error.
        self.num_epochs_trained += epochs_per_tick
        self.error_listbox.insert(tk.END, f'{self.num_epochs_trained}: {epoch_error:.8f}')
        self.error_listbox.see(tk.END)

        # Redraw the network.
        self.redraw_network()

        # Repeat in 10 milliseconds.
        if self.num_epochs_trained < max_epoch:
            self.window.after(10, self.tick, inputs, targets, learning_rate,
                max_epoch, epochs_per_tick)

In [16]:
App()